<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_1213_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Euler's Totient Function Values and partitions 的實驗
This feature content last 50 partitions of Euler's Totient Function Values <br>
leakyRelu batch_size = 1024<br>
no normallized


In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [11]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LayerNormalization
from random import choice, sample


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(primeTable, limit, addSize=5):
    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
      halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

def add_aim(index, staticFlagX):

  #feature = [index-flagN for flagN in staticFlagX] + []
  return [index-flagN for flagN in staticFlagX]

def add_mod(primeTable25, num):
  return [num%index for index in primeTable25]


def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def compute_coprimes(n, count=50): #反向搜尋coprime, 預設為100
    coprimes = []
    num = n
    while len(coprimes) < count and num >= 1:
        if gcd(num, n) == 1:
            coprimes.append(num%100)
        num -= 1

    if len(coprimes) < count:
        coprimes = coprimes + ([0] * (count - len(coprimes)))
    # return coprimes[::-1]  # 返回找到的質因數列表，這是反向的
    return coprimes

In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_1000.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [ ]:
print (shuffledNumbers[5])
print (shuffledLabel [5])

2110226
7596


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [ ]:
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:

    # Euler's Totient Function Partitions fist 50
    feature = []
    feature = compute_coprimes(num)
    #print (feature)
    mergedTrainNumber.append(feature)
    print(f"Euler's Totient Function of {num} calculated.")

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    print("merge number ", num, " done.")

#print (mergedTrainNumber)

In [ ]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
print (X_train[0])
print (X_train[50])
print (X_train[100])

[71 67 65 61 59 55 53 47 43 41 37 35 31 29 25 23 19 17 13 11  7  5  1 99
 95 93 89 87 83 81 77 75 71 69 65 63 59 53 51 47 45 41 39 35 33 29 27 23
 21 17 15  9  5  3 99 97 93 91 87 85 81 79 75 73 69 67 63 61 57 55 51 49
 45 43 39 37 33 31 27 25 21 15 13  9  7  3  1 97 95 91 89 85 83 79 77 71
 67 65 61 59]
[79 77 73 71 69 67 63 61 59 57 53 51 49 47 43 41 39 37 33 31 29 27 23 21
 19 17 13 11  9  7  3  1 99 97 93 91 89 87 83 81 79 77 73 71 69 67 63 61
 59 57 53 51 49 47 43 41 39 37 33 31 29 27 23 21 19 17 13 11  9  7  3  1
 99 97 93 91 89 87 83 81 79 77 73 71 69 67 63 61 59 57 53 51 49 47 43 41
 39 37 33 31]
[91 87 81 79 75 73 69 67 63 61 55 51 49 45 39 37 33 31 27 25 21 19 13  9
  7  3 97 95 91 89 85 83 79 77 71 67 65 61 55 53 49 47 43 41 37 35 29 25
 23 19 13 11  7  5  1 99 95 93 87 83 81 77 71 69 65 63 59 57 53 51 45 41
 39 35 29 27 23 21 17 15 11  9  3 99 97 93 87 85 81 79 75 73 69 67 61 57
 55 51 45 43]


In [ ]:


# start training
# 清空所有
keras.clear_session()

################################################################

# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    model = Sequential()

    model.add(Dense(units=100, input_shape=(100,), activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    print(model.summary())

    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # train model
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=200,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    model.save('./csv/goldbach_model_1112_400w.h5')
print("model saved.")


## predict

In [1]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [4]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [5]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_1211_400w_ep5000.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values

print("---loading test data done.---")


---loading test data done.---


In [6]:
# shuffle training data
shuffleIndices = np.random.permutation(len(testDataNumbers))
shuffledNumbers = testDataNumbers[shuffleIndices]
shuffledLabel = testDataLabel[shuffleIndices]

In [7]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [13]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []
staticFlagX = [316978, 2652438, 3897236, 3226506, 3571052, 3276150]
staticFlagY = [1577, 20090, 15400, 22108, 11879, 29621]

for num in shuffledNumbers:

   # Euler's Totient Function Partitions fist 50
    feature = []
    feature = compute_coprimes(num)
    #print (feature)
    mergedTestNumber.append(feature)
    print(f"Euler's Totient Function of {num} calculated.")

for num in shuffledLabel:
    mergedTestLabel.append(num)

    print("merge number ", num, " is done.")

#print (mergedTrainNumber)

Euler's Totient Function of 4561362 calculated.
Euler's Totient Function of 4324986 calculated.
Euler's Totient Function of 4653712 calculated.
Euler's Totient Function of 4010118 calculated.
Euler's Totient Function of 4215008 calculated.
Euler's Totient Function of 4306378 calculated.
Euler's Totient Function of 4291942 calculated.
Euler's Totient Function of 4833296 calculated.
Euler's Totient Function of 4170488 calculated.
Euler's Totient Function of 4604856 calculated.
Euler's Totient Function of 4347996 calculated.
Euler's Totient Function of 4634032 calculated.
Euler's Totient Function of 4166062 calculated.
Euler's Totient Function of 4550968 calculated.
Euler's Totient Function of 4704764 calculated.
Euler's Totient Function of 4867410 calculated.
Euler's Totient Function of 4920400 calculated.
Euler's Totient Function of 4948220 calculated.
Euler's Totient Function of 4234636 calculated.
Euler's Totient Function of 4790492 calculated.
Euler's Totient Function of 4158576 calc

In [14]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [15]:
evaluation = model.evaluate(mergedArray, mergedLabel)
#XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)

2/2 [==============================] - 0s 12ms/step - loss: 767845824.0000 - accuracy: 0.0000e+00
Loss: 767845824.0
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 7ms/step


In [16]:
print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataLabel[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")


測試五十筆資料
第0筆資料為數字:	4234636	|正確答案為:16566	模型預估為[-15649.003], 四捨五入到[-15649.]
第1筆資料為數字:	4010118	|正確答案為:31717	模型預估為[-15484.219], 四捨五入到[-15484.]
第2筆資料為數字:	4291942	|正確答案為:14594	模型預估為[22810.996], 四捨五入到[22811.]
第3筆資料為數字:	4151108	|正確答案為:14832	模型預估為[18999.504], 四捨五入到[19000.]
第4筆資料為數字:	4914148	|正確答案為:15673	模型預估為[18634.645], 四捨五入到[18635.]
第5筆資料為數字:	4634032	|正確答案為:16346	模型預估為[7609.2183], 四捨五入到[7609.]
第6筆資料為數字:	4010462	|正確答案為:13108	模型預估為[-9486.607], 四捨五入到[-9487.]
第7筆資料為數字:	4665262	|正確答案為:18098	模型預估為[4932.988], 四捨五入到[4933.]
第8筆資料為數字:	4874538	|正確答案為:31284	模型預估為[5209.1206], 四捨五入到[5209.]
第9筆資料為數字:	4867410	|正確答案為:42050	模型預估為[-15743.421], 四捨五入到[-15743.]
第10筆資料為數字:	4769998	|正確答案為:15623	模型預估為[-3479.1917], 四捨五入到[-3479.]
第11筆資料為數字:	4550968	|正確答案為:15886	模型預估為[6283.0386], 四捨五入到[6283.]
第12筆資料為數字:	4324986	|正確答案為:28364	模型預估為[-642.6409], 四捨五入到[-643.]
第13筆資料為數字:	4306378	|正確答案為:14047	模型預估為[3748.0037], 四捨五入到[3748.]
第14筆資料為數字:	4657756	|正確答案為:14997	模型預估為[4245.937], 四捨五入到[4246.]
第15筆資料為數字:	4948220	|正確答案為:22961	模型預估為[-15687

In [17]:
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

	正確答案   	/   	模型預測
0	16566  		/  	[-15649.] = [-1.058598] 
1	31717  		/  	[-15484.] = [-2.0483725] 
2	14594  		/  	[22811.] = [0.63977903] 
3	14832  		/  	[19000.] = [0.7806316] 
4	15673  		/  	[18635.] = [0.84105176] 
5	16346  		/  	[7609.] = [2.1482456] 
6	13108  		/  	[-9487.] = [-1.3816803] 
7	18098  		/  	[4933.] = [3.6687615] 
8	31284  		/  	[5209.] = [6.0057592] 
9	42050  		/  	[-15743.] = [-2.6710284] 
10	15623  		/  	[-3479.] = [-4.4906583] 
11	15886  		/  	[6283.] = [2.52841] 
12	28364  		/  	[-643.] = [-44.111977] 
13	14047  		/  	[3748.] = [3.7478654] 
14	14997  		/  	[4246.] = [3.53203] 
15	22961  		/  	[-15687.] = [-1.463696] 
16	16070  		/  	[4779.] = [3.3626282] 
17	13302  		/  	[14897.] = [0.89293146] 
18	13996  		/  	[6900.] = [2.028406] 
19	15871  		/  	[4162.] = [3.8133109] 
20	15845  		/  	[-20504.] = [-0.77277607] 
21	32717  		/  	[-16196.] = [-2.0200667] 
22	31204  		/  	[-7688.] = [-4.058793] 
23	19410  		/  	[-2864.] = [-6.7772346] 
24	30918  		/  	[-2229.] = [

In [21]:
import math
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse

# 使用 LN
mse_result = calculate_mse(roundPredictions, testDataLabel)

# 使用 MLP(old)
#mse_result = calculate_mse(predictions, testDataPartition)

print("均方誤差 (MSE):", mse_result)

mean = testData['Factors'].mean()

#+N MSE
MSE = int(calculate_mse(roundPredictions, testDataLabel))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

均方誤差 (MSE): [7.721485e+08]
MSE : 772148480
RMSE : 27787.55980650334
Error Rate : 1.3208572021085823  ->  132.08572%


<ipython-input-21-bc90de9b1501>:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(roundPredictions, testDataLabel))
